Collect data from OneDrive folder by date

In [1]:
import os
import numpy as np
import configparser as cp

import datetime

fpath = r"C:\Develop\vsc\sma\raw\2023-06-14"
fmeta = [[f,[os.path.join(os.path.join(fpath,f),m) for m in os.listdir(os.path.join(fpath,f)) if m.endswith(".meta")]] for f in os.listdir(fpath) if os.path.isdir(os.path.join(fpath,f))]
fnames = [(m[0],m[1][0]) for m in fmeta if len(m[1])>0]

sdate = datetime.datetime.strptime("2023-06-14","%Y-%m-%d").date()

def ini_date(fname):
  rc = None
  conf = None
  try:
      conf = cp.ConfigParser(interpolation=None)
      conf.read(fname, encoding="utf8")
      rc = datetime.datetime.strptime(conf["image"]["stamp"],"%Y-%m-%d %H:%M:%S").date()            
  except cp.ParsingError as argument:            
      conf.clear()
  return rc
  
ffolders = [(fid,ini_date(fname),fname) for fid,fname in fnames] 
ffolders = [[fid,fname] for fid,fdate,fname in ffolders if fdate==sdate]

print("folders",len(ffolders),"at",sdate,"with files of",len(fnames))

folders 15 at 2023-06-14 with files of 28


Request meta data for every folder by
<ul>
<li>concentration</li>
<li>reagent</li>
<li>stamp</li>
</ul>

In [7]:
def ini_meta(fname):
  rc = None
  conf = None
  try:
      
      conf = cp.ConfigParser(interpolation=None)
      conf.read(fname, encoding="utf8")

      con = conf["controls"]["concentration"]
      for s,t in [(">",""),(",","."),("?","0")]: con = con.replace(s,t)
      rea = conf["controls"]["reagent"]
      dat = datetime.datetime.strptime(conf["image"]["stamp"],"%Y-%m-%d %H:%M:%S").date()

      flo = np.round(float(con),6)
      rc = [rea,flo,str(flo),dat]

  except cp.ParsingError as argument:            
      conf.clear()
  return rc

fmeta = [ini_meta(fname)+[fid,fname] for fid,fname in ffolders]

In [9]:
import pandas as pd
fframes = pd.DataFrame(fmeta,columns=["rea","flo","slo","dat","id","info"]).sort_values(by=["rea","flo"]).reset_index(drop=True)

Copy files from OneDrive folders to data folder by 

|--data<br>
&nbsp;&nbsp;|--stamp<br>
&nbsp;&nbsp;&nbsp;&nbsp;|--reagent

In [10]:
import shutil

fdata = os.path.join(os.getcwd(),"data")

for index,row in fframes.iterrows():

  fsource = os.path.join(fpath,row["id"])  
  ftarget = os.path.join(fdata,str(row["dat"]),row["rea"],row["slo"])
  finfo = row["info"]
  
  if not os.path.exists(ftarget): os.makedirs(ftarget)  
  
  fnames = [os.path.join(fsource,fname) for fname in os.listdir(fsource) if os.path.isfile(os.path.join(fsource,fname)) & fname.endswith(".jpg")]

  if len(fnames)==0:
    continue
  
  shutil.copy2(finfo,ftarget)

  for fname in fnames: 
    shutil.copy2(fname,ftarget)